# ImageNet Classification
>Performance analysis for ImageNet Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [ImageNet,Rooflines,Performance Prediction]
- image: images/imagenet_logo.png

In [29]:
#hide
import pandas as pd
import numpy as np
import altair as alt

#import utils
#import scripts

#from scripts.utils import replace_data_df

W = 600
H = 480
pd.options.display.float_format = '{:20,.6f}'.format
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000

csv_path = "./data/cleaned_csv/backup.csv"

In [30]:
#hide
# master_df.loc[(master_df.NN_Topology =='RN50') & (master_df.PruningFactor != '100%')]

# Theoretical Analysis of ImageNet

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [105]:
#hide_input
# Run the Rooflines script which processes the data and creates the chart
%run scripts/altair_plots.py
rooflines('imagenet')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for ImageNet classification. The metric used for the theoretical performance is input/second.
Looking at the plot, it becomes clear that prunning along with quantization outputs some of the best performance results.

In [32]:
#hide_input
%run scripts/altair_plots.py
heatmap('data/performance_predictions_imagenet_mnist_cifar.csv', 'imagenet', 'Performance Prediction for ImageNet')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for ImageNet

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [33]:
#hide
print(pd.read_csv('data/overview_experiments_imagenet.csv').to_markdown())

|    | Hardware   | Platform         | ResNet50                  | GoogLeNetV1   | MobileNet   | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:--------------------------|:--------------|:------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | [INT8]*[100%,80%,50%,30%] | INT8          | na          | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | INT8                      | INT8          | na          | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | [INT8]*[100%,80%,50%,30%] | INT8          | INT8        | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | na                        | na            | na          | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | na                        | na            | na          | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | FP16,FP32

In [34]:
#hide_input
%run scripts/script_tables.py 
#get table with the experiments overview
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_imagenet_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [45]:
#hide
master_df = pd.read_csv(csv_path)
#fix ResNet50 Pruning values from 100,50,25,12.5 to -> 100,80,50,30
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp" or row["Op mode"] == "fast" or row["Op mode"] == "slow"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
imagenet_df = maxp_df[maxp_df.NN_Topology.isin(['GoogLeNetv1','ResNet-50','MobileNetv1','ResNet-50v15']) & maxp_df['lat-comp'].notna()]
bad_precisions = ["FP"+str(i) for i in range(17,24)]
#this version has the values for ResNet50 v1.5
imagenet_df.Datatype = imagenet_df.Datatype.apply(lambda x: 'FP16' if x in bad_precisions else x)
imagenet_df["hw_datatype_prun_net"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor, r.NN_Topology]), axis=1)

imagenet_df["PruningFactor"] = imagenet_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(imagenet_df, "lat-comp", "NN_Topology");
imagenet_df["datatype_model"] = imagenet_df.Datatype + '_' + imagenet_df.HWType
imagenet_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
imagenet_df["tag"] = imagenet_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.NN_Topology, str(r.PruningFactor)]), axis=1)

#filling GOPS data gaps 
imagenet_df['GOPS'] = imagenet_df.apply(lambda r: 1.14 if r.NN_Topology == 'MobileNetv1' else 
                                          ( 3.13 if r.NN_Topology == 'GoogLeNetv1' else 
                                           ( 8.2 if r.NN_Topology == 'ResNet-50v15' else 
                                            ( 7.72 if r.NN_Topology == 'ResNet-50' and r.PruningFactor == 100 else 
                                             ( 6.54 if r.NN_Topology == 'ResNet-50' and r.PruningFactor == 80 else 
                                              ( 3.75 if r.NN_Topology == 'ResNet-50' and r.PruningFactor == 50 else 
                                               ( 2.45 if r.NN_Topology == 'ResNet-50' and r.PruningFactor == 30 else 0 )))))) , axis=1)

#fill in tp-system and tp-cmp
imagenet_df['tp-system'] = imagenet_df['fps-system'] * imagenet_df['GOPS']
imagenet_df['tp-comp'] = imagenet_df['fps-comp'] * imagenet_df['GOPS']
imagenet_df['GOPS'] = imagenet_df['GOPS'] * imagenet_df['batch/thread/stream']
imagenet_df.head(300)

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_peak_perf,hw_bandwidth,nn_total_operations,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
0,MobileNetv1,ZCU104-DPU,INT8,NaN,8,38.08276832,33.807596,210.130758,236.642053,239.549064,269.771940,68.130000,87.626000,nan,15.720000,17.600000,9.120000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,1.000000,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
1,MobileNetv1,ZCU104-DPU,INT8,NaN,7,33.93144965,29.643530,206.410323,236.156816,235.307768,269.218770,68.130000,87.626000,nan,15.720000,17.500000,7.980000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.876830,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
2,MobileNetv1,ZCU104-DPU,INT8,NaN,6,29.6451959,25.482758,202.503510,235.470347,230.854001,268.436196,68.130000,87.626000,nan,15.720000,17.600000,6.840000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.753758,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
3,MobileNetv1,ZCU104-DPU,INT8,NaN,5,25.50485432,21.313983,196.202459,234.603609,223.670803,267.448114,68.130000,87.626000,nan,15.720000,17.600000,5.700000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.630450,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
4,MobileNetv1,ZCU104-DPU,INT8,NaN,4,21.26583904,17.127269,188.234089,233.575940,214.586862,266.276572,68.130000,87.626000,nan,15.720000,17.600000,4.560000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.506610,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
5,MobileNetv1,ZCU104-DPU,INT8,NaN,3,17.0441255,12.928530,176.205388,232.067764,200.874143,264.557251,68.130000,87.626000,nan,15.720000,17.600000,3.420000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.382415,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
6,MobileNetv1,ZCU104-DPU,INT8,NaN,2,13.25202793,8.763307,151.193536,228.277431,172.360631,260.236271,68.130000,87.626000,nan,15.720000,17.500000,2.280000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.259211,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
7,MobileNetv1,ZCU104-DPU,INT8,NaN,1,8.875291348,4.443876,113.045224,225.045547,128.871555,256.551923,68.130000,87.626000,nan,15.720000,17.600000,1.140000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.131446,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
8,ResNet-50v15,EdgeTPU,INT8,fast,1,NaN,40.441510,10.552000,24.727000,86.526400,202.761400,nan,nan,nan,nan,1.190000,8.200000,100.000000,l3,4.000000,25.600000,"10,000,000.000000",EdgeTPU_INT8_100%_ResNet-50v15,0.954757,INT8_EdgeTPU,EdgeTPU_INT8_ResNet-50v15_100.0
9,ResNet-50v15,EdgeTPU,INT8,fast,1,NaN,40.585040,10.589000,24.640000,86.829800,202.048000,nan,nan,nan,nan,1.490000,8.200000,100.000000,l3,4.000000,25.600000,"10,000,000.000000",EdgeTPU_INT8_100%_ResNet-50v15,0.958145,INT8_EdgeTPU,EdgeTPU_INT8_ResNet-50v15_100.0


### Line Plot

In [36]:
#hide_input
dataframe = imagenet_df
sel = alt.selection_multi(fields=["hw_datatype_prun_net"], bind="legend")
fig25_dot = alt.Chart(dataframe).mark_point().encode(
    x='lat-comp',
    y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
    color=select_color(sel, 'hw_datatype_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)
fig25_line = alt.Chart(dataframe).mark_line().encode(
    x='lat-comp',
    y='fps-comp',
    color=select_color(sel, 'hw_datatype_prun_net:N'),
    tooltip=['Op mode', 'fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
)

fig = (fig25_dot+fig25_line).properties(
    title="Latency Versus Performance for Pruned and Quantized ImageNet Classifier Variants",
    width=W,
    height=H,
).add_selection(sel).interactive()

fig

alt.LayerChart(...)

### Boxplots

In [37]:
#hide_input 
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df, xaxis='PruningFactor', yaxis="lat-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Latency by Hardware/Framework and Pruning for ImageNet Classification")

alt.FacetChart(...)

In [38]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df, xaxis='PruningFactor', yaxis="fps-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Throughput by Hardware/Framework and Pruning for ImageNet Classification")

alt.FacetChart(...)

In [39]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=imagenet_df, xaxis='PruningFactor', yaxis="Full_Pwr_W", color_col= 'PruningFactor', facet_column='datatype_model' , title="Power Consumption by Hardware/Framework and Pruning for ImageNet Classification")

alt.FacetChart(...)

## Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [107]:
#hide
%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
# ResNet50 v15 does not have accuracy measurements yet, so it needs to be taken out
df_pareto_graph = imagenet_df[imagenet_df.NN_Topology != 'RN50V15']
pareto_graph(df= df_pareto_graph, 
             groupcol= 'tag', 
             xcol= 'fps-comp', 
             ycol= 'top1', 
             W= W, 
             H= H, 
             title= "ImageNet Cassification Design Space: Accuracy Versus Performance")


       NN_Topology       HWType Datatype Op mode  batch/thread/stream  \
0      MobileNetv1   ZCU104-DPU     INT8     NaN                    8   
1      MobileNetv1   ZCU104-DPU     INT8     NaN                    7   
2      MobileNetv1   ZCU104-DPU     INT8     NaN                    6   
3      MobileNetv1   ZCU104-DPU     INT8     NaN                    5   
4      MobileNetv1   ZCU104-DPU     INT8     NaN                    4   
5      MobileNetv1   ZCU104-DPU     INT8     NaN                    3   
6      MobileNetv1   ZCU104-DPU     INT8     NaN                    2   
7      MobileNetv1   ZCU104-DPU     INT8     NaN                    1   
8     ResNet-50v15      EdgeTPU     INT8    fast                    1   
9     ResNet-50v15      EdgeTPU     INT8    fast                    1   
10    ResNet-50v15      EdgeTPU     INT8    slow                    1   
11    ResNet-50v15      EdgeTPU     INT8    slow                    1   
281    MobileNetv1      EdgeTPU     INT8    slow   

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
#hide 
%run scripts/overlapped_pareto.py

df = process_measured_data(csv_filepath= 'data/cleaned_csv/experimental_data_mnist.csv')
df

# Theoretical Pareto and Measured Pareto Overlapped

In order to easily understand how accurate predictions were, an overlapping between the Theoretical Pareto Plot and Measured Pareto Plot was made. In the plot below we have both theoretical (orange) and measured (blue) pareto lines. All measured datapoins are represented as crosses and all theoretical datatpoins are represented as circles. Some theoretical datapoints don't have a measured matched datapoint and the same goes for the measured datapoints. The theoretical pareto curve is, as expected, on the right of the measured one, as predictions are sometimes different form measurements.

In [104]:
#hide_input
%run scripts/altair_plots.py
%run scripts/overlapped_pareto.py
get_overlapped_pareto('imagenet','Overlapped Pareto Plots Theoretical + Measured for ImageNet') 

alt.LayerChart(...)

# Efficiency Plot

In order to understand the gap between the theoretical predictions and what was measured, an efficiency bar-chart was created. The size of the bar reflects the absolute performance, whereby all theoretical predictions are shown in red, theoretical peak performance in blue, and all measured datapoints in orange. The orange bars are annotated with the efficiency achieved as a percentage of the predicted performance. Please note the logarithmic y-axis scale. The theoretical predictions take memory bottlenecks into account, as such measured performance can actually exceed the predicted result, in which case the percentage can be above 100%.

In [106]:
#hide
%run scripts/overlapped_pareto.py
imagenet_df_tmp = imagenet_df[imagenet_df.NN_Topology != 'RN50V15']
#imagenet_efficiency_df = get_peak_perf_gops_df(df_=imagenet_df_tmp) #takes the imagenet df and fills it with data for the 3rd bar - Theoretical Peak Performance
efficiency_plot(machine_learning_task= 'imagenet', title='Efficiency Plots for ImageNet')

['edgetpu-fast_int8_efficientnet-l_100'
 'edgetpu-fast_int8_efficientnet-m_100'
 'edgetpu-fast_int8_efficientnet-s_100'
 'edgetpu-fast_int8_googlenetv1_100' 'edgetpu-fast_int8_mobilenetv1_100'
 'edgetpu-fast_int8_resnet-50_100' 'edgetpu-slow_int8_efficientnet-l_100'
 'edgetpu-slow_int8_efficientnet-m_100'
 'edgetpu-slow_int8_efficientnet-s_100'
 'edgetpu-slow_int8_googlenetv1_100' 'edgetpu-slow_int8_mobilenetv1_100'
 'edgetpu-slow_int8_resnet-50_100' 'ncs_fp16_googlenetv1_100'
 'ncs_fp16_resnet-50_100' 'ncs_int8_googlenetv1_100'
 'tx2-maxn_fp16_googlenetv1_100' 'tx2-maxn_fp16_resnet-50_100'
 'tx2-maxn_fp32_googlenetv1_100' 'tx2-maxn_fp32_resnet-50_100'
 'tx2-maxp_fp16_googlenetv1_100' 'tx2-maxp_fp16_resnet-50_100'
 'tx2-maxp_fp32_googlenetv1_100' 'tx2-maxp_fp32_resnet-50_100'
 'tx2-maxq_fp16_googlenetv1_100' 'tx2-maxq_fp16_resnet-50_100'
 'tx2-maxq_fp32_googlenetv1_100' 'tx2-maxq_fp32_resnet-50_100'
 'ultra96-dpu_int8_efficientnet-l_100'
 'ultra96-dpu_int8_efficientnet-m_100'
 'ultra96

IndexError: index 0 is out of bounds for axis 0 with size 0

In [46]:
#hide
imagenet_df.to_csv('data/cleaned_csv/experimental_data_imagenet.csv', index = False)
imagenet_df

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_peak_perf,hw_bandwidth,nn_total_operations,hw_datatype_prun_net,norm-lat-comp,datatype_model,tag
0,MobileNetv1,ZCU104-DPU,INT8,NaN,8,38.08276832,33.807596,210.130758,236.642053,239.549064,269.771940,68.130000,87.626000,nan,15.720000,17.600000,9.120000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,1.000000,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
1,MobileNetv1,ZCU104-DPU,INT8,NaN,7,33.93144965,29.643530,206.410323,236.156816,235.307768,269.218770,68.130000,87.626000,nan,15.720000,17.500000,7.980000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.876830,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
2,MobileNetv1,ZCU104-DPU,INT8,NaN,6,29.6451959,25.482758,202.503510,235.470347,230.854001,268.436196,68.130000,87.626000,nan,15.720000,17.600000,6.840000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.753758,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
3,MobileNetv1,ZCU104-DPU,INT8,NaN,5,25.50485432,21.313983,196.202459,234.603609,223.670803,267.448114,68.130000,87.626000,nan,15.720000,17.600000,5.700000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.630450,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
4,MobileNetv1,ZCU104-DPU,INT8,NaN,4,21.26583904,17.127269,188.234089,233.575940,214.586862,266.276572,68.130000,87.626000,nan,15.720000,17.600000,4.560000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.506610,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
5,MobileNetv1,ZCU104-DPU,INT8,NaN,3,17.0441255,12.928530,176.205388,232.067764,200.874143,264.557251,68.130000,87.626000,nan,15.720000,17.600000,3.420000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.382415,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
6,MobileNetv1,ZCU104-DPU,INT8,NaN,2,13.25202793,8.763307,151.193536,228.277431,172.360631,260.236271,68.130000,87.626000,nan,15.720000,17.500000,2.280000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.259211,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
7,MobileNetv1,ZCU104-DPU,INT8,NaN,1,8.875291348,4.443876,113.045224,225.045547,128.871555,256.551923,68.130000,87.626000,nan,15.720000,17.600000,1.140000,100.000000,l3,4.600000,19.200000,1.100000,ZCU104-DPU_INT8_100%_MobileNetv1,0.131446,INT8_ZCU104-DPU,ZCU104-DPU_INT8_MobileNetv1_100.0
8,ResNet-50v15,EdgeTPU,INT8,fast,1,NaN,40.441510,10.552000,24.727000,86.526400,202.761400,nan,nan,nan,nan,1.190000,8.200000,100.000000,l3,4.000000,25.600000,"10,000,000.000000",EdgeTPU_INT8_100%_ResNet-50v15,0.954757,INT8_EdgeTPU,EdgeTPU_INT8_ResNet-50v15_100.0
9,ResNet-50v15,EdgeTPU,INT8,fast,1,NaN,40.585040,10.589000,24.640000,86.829800,202.048000,nan,nan,nan,nan,1.490000,8.200000,100.000000,l3,4.000000,25.600000,"10,000,000.000000",EdgeTPU_INT8_100%_ResNet-50v15,0.958145,INT8_EdgeTPU,EdgeTPU_INT8_ResNet-50v15_100.0
